In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ev = pd.read_csv("known_exploited_vulnerabilities.csv")
ev.head()

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,notes,cwes
0,CVE-2026-21513,Microsoft,Windows,Microsoft MSHTML Framework Protection Mechanis...,2/10/2026,Microsoft MSHTML Framework contains a protecti...,"Apply mitigations per vendor instructions, fol...",3/3/2026,Unknown,https://msrc.microsoft.com/update-guide/adviso...,CWE-693
1,CVE-2026-21525,Microsoft,Windows,Microsoft Windows NULL Pointer Dereference Vul...,2/10/2026,Microsoft Windows Remote Access Connection Man...,"Apply mitigations per vendor instructions, fol...",3/3/2026,Unknown,https://msrc.microsoft.com/update-guide/vulner...,CWE-476
2,CVE-2026-21510,Microsoft,Windows,Microsoft Windows Shell Protection Mechanism F...,2/10/2026,Microsoft Windows Shell contains a protection ...,"Apply mitigations per vendor instructions, fol...",3/3/2026,Unknown,https://msrc.microsoft.com/update-guide/vulner...,CWE-693
3,CVE-2026-21533,Microsoft,Windows,Microsoft Windows Improper Privilege Managemen...,2/10/2026,Microsoft Windows Remote Desktop Services cont...,"Apply mitigations per vendor instructions, fol...",3/3/2026,Unknown,https://msrc.microsoft.com/update-guide/vulner...,CWE-269
4,CVE-2026-21519,Microsoft,Windows,Microsoft Windows Type Confusion Vulnerability,2/10/2026,Microsoft Desktop Windows Manager contains a t...,"Apply mitigations per vendor instructions, fol...",3/3/2026,Unknown,https://msrc.microsoft.com/update-guide/vulner...,CWE-843


In [3]:
ev.shortDescription

0       Microsoft MSHTML Framework contains a protecti...
1       Microsoft Windows Remote Access Connection Man...
2       Microsoft Windows Shell contains a protection ...
3       Microsoft Windows Remote Desktop Services cont...
4       Microsoft Desktop Windows Manager contains a t...
                              ...                        
1508    Yealink Device Management contains a server-si...
1509    Zoho ManageEngine ADSelfService Plus contains ...
1510    Zoho ManageEngine Desktop Central contains a f...
1511    Zoho ManageEngine ServiceDesk Plus (SDP) conta...
1512    Zyxel firewalls (ATP, USG, VM) and AP Controll...
Name: shortDescription, Length: 1513, dtype: object

In [4]:
new = pd.read_csv("ev_extracted.csv")
new

,shortDescription,company,vuln_type
0,Microsoft MSHTML Framework contains a protecti...,Microsoft MSHTML Framework,NaN
1,Microsoft Windows Remote Access Connection Man...,Microsoft Windows Remote Access,NaN
2,Microsoft Windows Shell contains a protection ...,Microsoft Windows Shell,NaN
3,Microsoft Windows Remote Desktop Services cont...,Microsoft Windows Remote Desktop,NaN
4,Microsoft Desktop Windows Manager contains a t...,Microsoft Desktop Windows Manager,NaN
...,...,...,...
1508,Yealink Device Management contains a server-si...,Yealink Device Management,Remote Code Execution
1509,Zoho ManageEngine ADSelfService Plus contains ...,Zoho ManageEngine ADSelfService Plus,Remote Code Execution
1510,Zoho ManageEngine Desktop Central contains a f...,Zoho ManageEngine Desktop Central,Remote Code Execution
1511,Zoho ManageEngine ServiceDesk Plus (SDP) conta...,Zoho ManageEngine ServiceDesk Plus,remote users to upload files via login page cu...
